In [1]:
import cv2

In [2]:
import numpy as np

In [7]:
def apply_dog_filter(image, blur_size=(0,0), k_factor=1.6, sigma_value=0.5, gamma_value=1):
    blurred_image_1 = cv2.GaussianBlur(image, blur_size, sigma_value)
    blurred_image_2 = cv2.GaussianBlur(image, blur_size, sigma_value * k_factor)
    return (blurred_image_1 - gamma_value * blurred_image_2)

In [8]:
def apply_xdog_filter(image, sigma_value=0.5, k_factor=1.6, gamma_value=1, threshold=1, sharpness=1):
    dog_output = apply_dog_filter(image, sigma_value=sigma_value, k_factor=k_factor, gamma_value=gamma_value) / 255
    xdog_result = np.zeros_like(dog_output)
    for row in range(dog_output.shape[0]):
        for col in range(dog_output.shape[1]):
            if dog_output[row, col] < threshold:
                dog_output[row, col] = 255
            else:
                dog_output[row, col] = 255 * (1 + np.tanh(sharpness * (dog_output[row, col])))
    return dog_output

In [9]:
def convert_to_line_art(original_image, sigma_value=0.8, k_factor=5, gamma_value=0.95, epsilon=-1, phi=10e15, min_area=2):
    img_cnts = []
    grayscale_image = cv2.cvtColor(np.array(original_image), cv2.COLOR_RGB2GRAY)
    xdog_image = apply_xdog_filter(grayscale_image, sigma_value=sigma_value, k_factor=k_factor, gamma_value=gamma_value, threshold=epsilon, sharpness=phi).astype(np.uint8)
    blank_canvas = np.zeros_like(xdog_image)
    binary_thresh = cv2.threshold(xdog_image, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    contours = cv2.findContours(binary_thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    for contour in contours:
        if cv2.contourArea(contour) > min_area:
            img_cnts.append(contour)

    return Image.fromarray(255 - cv2.drawContours(blank_canvas, img_cnts, -1, (255,255,255), -1))

In [11]:
import cv2
from PIL import Image


In [12]:
def to_gray(image_path):
    
    img = cv2.imread(image_path)

    # Convert from BGR to RGB
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Convert to PIL Image
    img_pil = Image.fromarray(img)

    # Assuming 'to_sketch' is a function you have defined to convert the image to a sketch
    sketch_result = convert_to_line_art(img_pil)
    
    return sketch_result

In [15]:
import os
import cv2

dataset_path = "DATASET/"
output_path = "linearts/"

# Create the output directory if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Iterate over all image files in the dataset directory
for filename in os.listdir(dataset_path):
    if filename.lower().endswith((".png", ".jpg", ".jpeg")):  # Check if the file is an image
        # Construct the full path to the image file
        image_path = os.path.join(dataset_path, filename)
        
        # Convert the image to a sketch
        sketch_result = to_gray(image_path)
        
        # Construct the output filename and path
        output_filename = os.path.splitext(filename)[0] + ".jpg"  # Change extension to .jpg
        output_file_path = os.path.join(output_path, output_filename)
        
        # Save the result
        sketch_result.save(output_file_path)

print("Conversion completed and saved to", output_path)


Conversion completed and saved to linearts/
